# Importar ferramentas

In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from itertools import chain
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\Protege
[nltk_data]     Auto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Protege
[nltk_data]     Auto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Carregar conjuntos de dados

In [2]:
artistas = pd.read_csv("C:/Users/Protege Auto/Documents/ENEM GABRIEL/artists-data.csv")
letras = pd.read_csv('C:/Users/Protege Auto/Documents/ENEM GABRIEL/lyrics-data.csv')

In [3]:
artistas.head(10)

,Artist,Genres,Songs,Popularity,Link
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/
5,Harmonia do Samba,Axé; Samba; Pagode,237.0,0.9,/harmonia-do-samba/
6,Ara Ketu,Axé; Pop,139.0,1.5,/ara-ketu/
7,Daniela Mercury,MPB; Axé,230.0,1.4,/daniela-mercury/
8,Olodum,Axé,74.0,1.3,/olodum/
9,Netinho,Axé,204.0,2.0,/netinho/


In [4]:
letras.head(10)

,ALink,SName,SLink,Lyric,language
0,/ivete-sangalo/,Arerê,/ivete-sangalo/arere.html,"Tudo o que eu quero nessa vida,\nToda vida, é\...",pt
1,/ivete-sangalo/,Se Eu Não Te Amasse Tanto Assim,/ivete-sangalo/se-eu-nao-te-amasse-tanto-assim...,Meu coração\nSem direção\nVoando só por voar\n...,pt
2,/ivete-sangalo/,Céu da Boca,/ivete-sangalo/chupa-toda.html,É de babaixá!\nÉ de balacubaca!\nÉ de babaixá!...,pt
3,/ivete-sangalo/,Quando A Chuva Passar,/ivete-sangalo/quando-a-chuva-passar.html,Quando a chuva passar\n\nPra quê falar\nSe voc...,pt
4,/ivete-sangalo/,Sorte Grande,/ivete-sangalo/sorte-grande.html,A minha sorte grande foi você cair do céu\nMin...,pt
5,/ivete-sangalo/,A Lua Q Eu T Dei,/ivete-sangalo/a-lua-q-eu-t-dei.html,"Posso te falar dos sonhos, das flores...\nde c...",pt
6,/ivete-sangalo/,Mulheres Não Têm Que Chorar (com Emicida),/ivete-sangalo/mulheres-nao-tem-que-chorar-com...,"Hey, girl\nLevanta da cama\nO que foi que te a...",pt
7,/ivete-sangalo/,Eva / Alô Paixão / Beleza Rara,/ivete-sangalo/eva-alo-paixao-beleza-rara.html,"""EVA""\n(Giancarlo Bigazzi/Umberto Tozzi)\n\nMe...",pt
8,/ivete-sangalo/,Flor do Reggae,/ivete-sangalo/flor-do-reggae.html,Um brilho de amor chegou na ilha inteira\nE a ...,pt
9,/ivete-sangalo/,Carro Velho,/ivete-sangalo/carro-velho.html,"Cheiro de pneu queimado, carburador furado, co...",pt


# Criação do conjunto de dados de trabalho

In [5]:
artistas['Genres'].unique() # Listar gêneros musicais presentes no dataset

array(['Pop; Axé; Romântico', 'Axé', 'Axé; Romântico; Reggae', ...,
       'World Music; Indie; New Age', 'World Music; Gospel/Religioso',
       'World Music; Black Music; Blues'], dtype=object)

## Vários gêneros são compostos, é preciso separá-los

In [6]:
# Cria lista de listas com os valores do atributo.
# Assim, um valor composto acaba decomposto em uma lista cujos elementos
# são cada um dos gêneros principais
lista_generos = [str(s).split(';') for s in artistas['Genres'].unique()] 
lista_generos

[['Pop', ' Axé', ' Romântico'],
 ['Axé'],
 ['Axé', ' Romântico', ' Reggae'],
 ['Axé', ' Samba', ' Pagode'],
 ['Axé', ' Pop'],
 ['MPB', ' Axé'],
 ['Axé', ' Forró', ' Romântico'],
 ['Axé', ' Romântico', ' Chillout'],
 ['Romântico', ' Axé', ' Reggae'],
 ['Axé', ' Romântico', ' Trilha Sonora'],
 ['Axé', ' Samba', ' MPB'],
 ['Axé', ' Pop', ' Romântico'],
 ['Axé', ' Pagode', ' Samba'],
 ['Axé', ' Pagode', ' Funk'],
 ['Pagode', ' Axé', ' Funk'],
 ['Axé', ' Romântico', ' Pagode'],
 ['Axé', ' Dance', ' Pop/Rock'],
 ['Axé', ' Funk', ' Electronica'],
 ['Axé', ' Romântico', ' Dance'],
 ['Pop', ' Axé', ' Sertanejo'],
 ['Pagode', ' Axé', ' Samba'],
 ['Funk Carioca', ' Axé', ' Funk'],
 ['Axé', ' Funk', ' Forró'],
 ['Axé', ' Pagode'],
 ['Sertanejo', ' MPB', ' Axé'],
 ['World Music', ' Axé', ' MPB'],
 ['MPB', ' Pop', ' Axé'],
 ['Pop', ' R&B', ' Black Music'],
 ['R&B', ' Pop', ' Black Music'],
 ['Hip Hop', ' Rap', ' Black Music'],
 ['R&B', ' Black Music', ' Soul Music'],
 ['Black Music', ' Soul Music', 

In [7]:
listas_unidas = list(chain(*lista_generos)) # Juntar as sublistas criadas
listas_unidas = [genero.strip() for genero in listas_unidas] # Tirar os espaços no começo e final das palavras
listas_unidas

['Pop',
 'Axé',
 'Romântico',
 'Axé',
 'Axé',
 'Romântico',
 'Reggae',
 'Axé',
 'Samba',
 'Pagode',
 'Axé',
 'Pop',
 'MPB',
 'Axé',
 'Axé',
 'Forró',
 'Romântico',
 'Axé',
 'Romântico',
 'Chillout',
 'Romântico',
 'Axé',
 'Reggae',
 'Axé',
 'Romântico',
 'Trilha Sonora',
 'Axé',
 'Samba',
 'MPB',
 'Axé',
 'Pop',
 'Romântico',
 'Axé',
 'Pagode',
 'Samba',
 'Axé',
 'Pagode',
 'Funk',
 'Pagode',
 'Axé',
 'Funk',
 'Axé',
 'Romântico',
 'Pagode',
 'Axé',
 'Dance',
 'Pop/Rock',
 'Axé',
 'Funk',
 'Electronica',
 'Axé',
 'Romântico',
 'Dance',
 'Pop',
 'Axé',
 'Sertanejo',
 'Pagode',
 'Axé',
 'Samba',
 'Funk Carioca',
 'Axé',
 'Funk',
 'Axé',
 'Funk',
 'Forró',
 'Axé',
 'Pagode',
 'Sertanejo',
 'MPB',
 'Axé',
 'World Music',
 'Axé',
 'MPB',
 'MPB',
 'Pop',
 'Axé',
 'Pop',
 'R&B',
 'Black Music',
 'R&B',
 'Pop',
 'Black Music',
 'Hip Hop',
 'Rap',
 'Black Music',
 'R&B',
 'Black Music',
 'Soul Music',
 'Black Music',
 'Soul Music',
 'R&B',
 'Electronica',
 'Black Music',
 'Hip Hop',
 'Romântico

In [8]:
generos = list(set(listas_unidas)) # Listar valores, sem repetição
generos

['Emocore',
 'Metal',
 'Piseiro',
 'Punk Rock',
 'Músicas Gaúchas',
 'Jazz',
 'Fado',
 'Rap',
 'Trilha Sonora',
 'Hip Hop',
 'Lo-fi',
 'Urban',
 'Rock',
 'Infantil',
 'Rock Alternativo',
 'nan',
 'Clássico',
 'Gospel/Religioso',
 'Pós-Punk',
 'Romântico',
 'Velha Guarda',
 'New Wave',
 'Folk',
 'Hardcore',
 'Reggaeton',
 'Hard Rock',
 'Power-Pop',
 'Pop',
 'MPB',
 'Ska',
 'Reggae',
 'J-Pop/J-Rock',
 'Instrumental',
 'Surf Music',
 'Funk Carioca',
 'Jovem Guarda',
 'Psicodelia',
 'Sertanejo',
 'New Age',
 'Soul Music',
 'Electronica',
 'Funk',
 'R&B',
 'Rockabilly',
 'Heavy Metal',
 'Country',
 'Forró',
 'Tecnopop',
 'Industrial',
 'Trap',
 'Regional',
 'Pop/Punk',
 'Samba',
 'Blues',
 'Black Music',
 'Pop/Rock',
 'Gótico',
 'Chillout',
 'Post-Rock',
 'Piano Rock',
 'Bossa Nova',
 'Dance',
 'Samba Enredo',
 'Trip-Hop',
 'Electro Swing',
 'Soft Rock',
 'House',
 'Pagode',
 'Classic Rock',
 'Axé',
 'Indie',
 'COLETÂNEA',
 'Tropical House',
 'Progressivo',
 'K-Pop/K-Rock',
 'Grunge',
 'Wor

In [9]:
# Transformar presença/ausencia de determinado genero em valor booleano,
# True, se o gênero é do artista
# False, caso contrário.
# Concatenar ao dataframe artistas
for genero in generos:
    todos_generos_em_bool = [True if re.search(genero, str(gen_artista)) else False for gen_artista in artistas['Genres']]
    artistas[genero] = todos_generos_em_bool

artistas.head()

,Artist,Genres,Songs,Popularity,Link,Emocore,Metal,Piseiro,Punk Rock,Músicas Gaúchas,...,Indie,COLETÂNEA,Tropical House,Progressivo,K-Pop/K-Rock,Grunge,World Music,Trance,Kizomba,Disco
0,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Chiclete com Banana,Axé,268.0,3.8,/chiclete-com-banana/,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Banda Eva,Axé; Romântico; Reggae,215.0,2.3,/banda-eva/,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,É O Tchan,Axé,129.0,1.6,/e-o-tchan/,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Claudia Leitte,Pop; Axé; Romântico,167.0,1.5,/claudia-leitte/,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Trabalhar só com as músicas em inglês e do tipo Country, pois estão presentes em maior quantidade

In [10]:
# Selecionar a coluna 'Link' dos registros de artistas que cantam música Country
artistas_country = artistas[artistas['Country'] == True]['Link'].unique()

# Selecionar letras no idioma inglês
musicas_ingles = letras[letras['language']=='en']

In [11]:
# Realizar junção dos datasets en_songs (contendo músicas em inglês), com o de artistas,
# usando como ponto de junção as colunas de "ALink" e "Link", que contém registros correspondentes 
dataset_unido = musicas_ingles.merge(artistas, how = 'outer', left_on='ALink', right_on = 'Link')

In [12]:
dataset_unido.head()

,ALink,SName,SLink,Lyric,language,Artist,Genres,Songs,Popularity,Link,...,Indie,COLETÂNEA,Tropical House,Progressivo,K-Pop/K-Rock,Grunge,World Music,Trance,Kizomba,Disco
0,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,...,False,False,False,False,False,False,False,False,False,False
1,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,...,False,False,False,False,False,False,False,False,False,False
2,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,...,False,False,False,False,False,False,False,False,False,False
3,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,...,False,False,False,False,False,False,False,False,False,False
4,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en,Ivete Sangalo,Pop; Axé; Romântico,313.0,4.4,/ivete-sangalo/,...,False,False,False,False,False,False,False,False,False,False


### Selecionar gênero a ser predito:

In [13]:
# A partir do dataset com todas as músicas em inglês, 
# selecionar dois conjuntos iguais, um de músicas country
# outro de músicas não country
nao_country = dataset_unido[dataset_unido['Country']==False].sample(n = 2500, random_state = 1)
country = dataset_unido[dataset_unido['Country']==True].sample(n = 2500, random_state = 1)

In [14]:
# criar dataframe balanceado,metade das músicas é country e a outra, de outros gêneros
dataframe_balanceado = pd.concat([nao_country,country]).reset_index()
dataframe_balanceado.head()

,index,ALink,SName,SLink,Lyric,language,Artist,Genres,Songs,Popularity,...,Indie,COLETÂNEA,Tropical House,Progressivo,K-Pop/K-Rock,Grunge,World Music,Trance,Kizomba,Disco
0,105195,/foster-the-people/,Houdini (RAC Remix),/foster-the-people/houdini-rac-remix.html,"Rise above, gonna start the war\nOh, what you ...",en,Foster The People,Indie; Rock Alternativo,66.0,10.8,...,True,False,False,False,False,False,False,False,False,False
1,152794,/magazine/,Twenty Years Ago,/magazine/twenty-years-ago.html,You turn pandemonium\ninto pantomime for one\n...,en,Magazine (UK),Pós-Punk,50.0,0.0,...,False,False,False,False,False,False,False,False,False,False
2,104322,/lana-del-rey/,Pretty When You Cry,/lana-del-rey/pretty-when-you-cry.html,"All the pretty stars shine for you, my love\nA...",en,Lana Del Rey,Romântico; Indie,311.0,53.3,...,True,False,False,False,False,False,False,False,False,False
3,15543,/gary-moore/,Speak for Yourself,/gary-moore/speak-for-yourself.html,(Gary Moore/Neil Carter)\nLook around across t...,en,Gary Moore,Blues; Rock; Jazz,167.0,0.0,...,False,False,False,False,False,False,False,False,False,False
4,74615,/rainbow/,Magic,/rainbow/magic.html,There's a light in the sky that hangs on the e...,en,Rainbow,Hard Rock; Heavy Metal; Rock,75.0,0.0,...,False,False,False,False,False,False,False,False,False,False


### Separação de variável preditora (letra) e variável-alvo (a classificação do gênero musical)

In [15]:
y = dataframe_balanceado['Country'].copy().astype(bool) # Variável-alvo, transformada em valores booleanos (1 e 0)
X = dataframe_balanceado.drop(['index', 'ALink', 'SName', 'SLink', 'language', 'Artist',
       'Genres', 'Songs', 'Popularity', 'Link', 'Axé', 'MPB', 'Tropical House',
       'Black Music', 'Piseiro', 'Dance', 'COLETÂNEA', 'Forró', 'Heavy Metal',
       'Pop/Rock', 'Soul Music', 'Metal', 'Reggae', 'House', 'Punk Rock',
       'Trilha Sonora', 'Electronica', 'Rock', 'Folk', 'Clássico',
       'Músicas Gaúchas', 'Lo-fi', 'Hip Hop', 'Tecnopop', 'nan', 'Gótico',
       'New Wave', 'Chillout', 'Rap', 'Classic Rock', 'Piano Rock',
       'Rock Alternativo', 'Romântico', 'Trip-Hop', 'Pop', 'Funk Carioca',
       'Funk', 'Industrial', 'Bossa Nova', 'Progressivo', 'Hard Rock',
       'J-Pop/J-Rock', 'Surf Music', 'Ska', 'World Music', 'New Age',
       'Emocore', 'Blues', 'Instrumental', 'Pós-Punk', 'Hardcore',
       'K-Pop/K-Rock', 'Kizomba', 'Jazz', 'Rockabilly', 'Trance', 'Disco',
       'Urban', 'R&B', 'Pop/Punk', 'Reggaeton', 'Trap', 'Psicodelia',
       'Country', 'Sertanejo', 'Samba', 'Samba Enredo', 'Power-Pop', 'Grunge',
       'Velha Guarda', 'Post-Rock', 'Gospel/Religioso', 'Infantil',
       'Jovem Guarda', 'Pagode', 'Electro Swing', 'Soft Rock', 'Fado',
       'Regional', 'Indie'], axis = 1)


In [16]:
X # Nota-se que as letras não estão limpas

,Lyric
0,"Rise above, gonna start the war\nOh, what you ..."
1,You turn pandemonium\ninto pantomime for one\n...
2,"All the pretty stars shine for you, my love\nA..."
3,(Gary Moore/Neil Carter)\nLook around across t...
4,There's a light in the sky that hangs on the e...
...,...
4995,I keep a close watch on this heart of mine\nI ...
4996,Hurt is no stranger to me\nAnd oh how well I k...
4997,You can let a gal with freckles on her face\nS...
4998,"You're fumbling around in the bedroom,\nTrying..."


In [17]:
y

0       False
1       False
2       False
3       False
4       False
        ...  
4995     True
4996     True
4997     True
4998     True
4999     True
Name: Country, Length: 5000, dtype: bool

# Tokenizar, remover stopwords e outras impurezas e vetorizar o dataset:
## Conceitos:

### Tokenização - decompõe uma string em um conjunto de tokens. Neste caso, separa a string em uma lista de palavras.

### Stopwords (ou palavras de parada) - palavras sem significado e irrelevantes, melhor removê-las.

###  Vetorização - essencialmente, converte dados de texto em dados numéricos, que são o tipo que o modelo consegue interpretar. Há várias técnicas, dentre as quais, a escolhida:

### TF-IDF, sigla composta que representa um método que une dois conceitos:

#### TF (Term Frequency) - cálculo da frequência de uma palavra, em relação ao total de palavras do documento:
##### Quanto mais vezes uma palavra aparecer em um documento, maior o valor de TF 
#### IDF (Inverse Document Frequency) - Inverso da Document Frequency
##### DF (Document Frequency): Frequência com que uma palavra está contida num documento, em relação ao total de documentos
##### A ideia de usar o inverso é que, quanto maior a frequencia de uma palavra no total dos documentos, menos importante ela é para um só
##### Uma palavra rara, então, aumenta o valor de IDF e,uma comum, diminui-o
##### Usa-se o log para diminuir a escala da razão IDF
### Se o valor de TF - IDF é o produto das partes, quanto maior os dois forem, mais o total vale
![1_1pTLnoOPJKKcKIcRi3q0WA.jpg](attachment:1_1pTLnoOPJKKcKIcRi3q0WA.jpg)

In [18]:
def tratar_letra(dataframe, coluna = 'Lyric'):
    # Ler todo o conteúdo da coluna como string, remover \n das strings
    letras = dataframe[coluna].astype(str).apply(lambda x: x.replace("\n", " "))
    caixa_baixa = [letra.lower() for letra in letras]
    letras_tokenizadas = [word_tokenize(letra) for letra in caixa_baixa] # Tokenizar letra

    # Criar conjunto contendo todas as palavras de parada da língua inglesa
    palavras_parada = set(stopwords.words('english'))

    # Loop para remover as palavras de parada do vetor tokenizado e inserir valores filtrados em lista
    letras_sem_stopwords = []
    for token in letras_tokenizadas: 
        letra_sem_stopwords = [palavra for palavra in token if palavra not in palavras_parada]
        letras_sem_stopwords.append(letra_sem_stopwords)

    # Remover caracteres não alfanuméricos
    sem_nao_alfanumericos = []
    for letra in letras_sem_stopwords:
        letra_limpa = [palavra for palavra in letra if palavra.isalpha()]
        sem_nao_alfanumericos.append(letra_limpa)
        
    # Lista de strings em que cada string é a letra da música tratada    
    letras_limpas = []
    for letra in sem_nao_alfanumericos:
        lista_em_string = ' '.join(letra)
        letras_limpas.append(lista_em_string)

    return letras_limpas

def vetorizar_dataset(dataset_treino, dataset_teste): 
    # Preparação do transformador de vetorização TF-IDF 
    vetorizador1 = TfidfVectorizer(min_df = 5, max_df = 0.8)
    vetores1 = vetorizador1.fit_transform(dataset_treino)
    nomes_features1 = vetorizador1.get_feature_names_out()
    dense1 = vetores1.todense()
    dense_list1 = dense1.tolist()
    letras_treino = pd.DataFrame(dense_list1, columns=nomes_features1)
    
    # agora, para o conjunto de teste, com o vocabulário do conjunto de treino
    vetorizador2 = TfidfVectorizer(vocabulary=vetorizador1.vocabulary_)
    vetores2 = vetorizador2.fit_transform(dataset_teste)
    nomes_features2 = vetorizador2.get_feature_names_out()
    dense2 = vetores2.todense()
    dense_list2 = dense2.tolist()
    letras_teste = pd.DataFrame(dense_list2, columns=nomes_features2)
    
    return letras_treino, letras_teste

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

X_train_limpo = tratar_letra(X_train)
X_test_limpo = tratar_letra(X_test)

In [20]:
X_train_pronto, X_test_pronto = vetorizar_dataset(X_train_limpo, X_test_limpo)

In [21]:
X_train_pronto.to_csv('X_train.csv', index = False)
y_train.to_csv('y_train.csv', index = False)

X_test_pronto.to_csv('X_test.csv', index = False)
y_test.to_csv('y_test.csv', index = False)